# Average Bubble

In [1]:
cd '/home/dpirvu/project/paper_prefactor/'

/home/dpirvu/project/paper_prefactor


In [2]:
import os,sys
sys.path.append('/home/dpirvu/python_stuff/')
sys.path.append('/home/dpirvu/project/paper_prefactor/bubble_codes/')
sys.path.remove('/home/dpirvu/DarkPhotonxunWISE/hmvec-master')
print(sys.path)
from plotting import *
from bubble_tools import *
from experiment import *
from celluloid import Camera
from matplotlib.colors import LogNorm

%matplotlib inline

['/home/dpirvu/project/paper_prefactor', '/cm/shared/apps/python/python37/lib/python37.zip', '/cm/shared/apps/python/python37/lib/python3.7', '/cm/shared/apps/python/python37/lib/python3.7/lib-dynload', '', '/home/dpirvu/.local/lib/python3.7/site-packages', '/cm/shared/apps/python/python37/lib/python3.7/site-packages', '/cm/shared/apps/python/python37/lib/python3.7/site-packages/IPython/extensions', '/home/dpirvu/.ipython', '/home/dpirvu/python_stuff/', '/home/dpirvu/project/paper_prefactor/bubble_codes/']
['/home/dpirvu/project/paper_prefactor', '/cm/shared/apps/python/python37/lib/python37.zip', '/cm/shared/apps/python/python37/lib/python3.7', '/cm/shared/apps/python/python37/lib/python3.7/lib-dynload', '', '/home/dpirvu/.local/lib/python3.7/site-packages', '/cm/shared/apps/python/python37/lib/python3.7/site-packages', '/cm/shared/apps/python/python37/lib/python3.7/site-packages/IPython/extensions', '/home/dpirvu/.ipython', '/home/dpirvu/python_stuff/', '/home/dpirvu/project/paper_pr

In [5]:
case = 'minus'
general = get_general_model(case)
tempList, massq, right_Vmax, V, dV, Vinv, nTimeMAX, minSim, maxSim = general

tmp = 2

maxSim = (1000 if tmp == 0 else 3000)
temp, m2, sigmafld = get_model(*general, tmp)
exp_params = [nLat, m2, temp]
print('Experiment', exp_params)

Experiment [2048, 0.8049999999999999, 0.13]


In [6]:
print('Looking at at T, m2, sigma:', temp, m2, sigmafld)

crit_radList    = np.array(np.linspace(5, 50, 10), dtype='int'); print(crit_radList)
crit_threshList = right_Vmax + np.linspace(0.5, 5, 10) * sigmafld
crit_threshList = np.array([round(ii, 3) for ii in crit_threshList]); print(crit_threshList)

win = 250
plots = False
get_data = True
get_averaged = True

if get_data:
    path = decay_times_file(*exp_params, minSim, maxSim, nTimeMAX)
    if os.path.exists(path):
        decay_times = np.load(path)

        minDecTime = 256
        alltimes   = decay_times[:,1]
        simList2Do = decay_times[alltimes>=minDecTime, 0]

    all_data = []
    for sim in range(minSim, maxSim):  
        path2RESTsim = rest_sim_location(*exp_params, sim)
        if os.path.exists(path2RESTsim):
            sim, bubble, totbeta = np.load(path2RESTsim, allow_pickle=True)

#             real = np.copy(bubble)
#             real = np.abs(real[0])
#             real = gaussian_filter(real, 1, mode='nearest')
#             nT, nN = np.shape(real)
#             tcen, xcen = find_nucleation_center(real, phieq, 2.7, 30)
#             t, x = np.linspace(-tcen, nT-1-tcen, nT), np.linspace(-xcen, nN-1-xcen, nN)

#             test = bubble[0, tcen, xcen-50:xcen+50]
#             if np.nanmean(test) < 0:
#                 bubble = - bubble

            all_data.append(np.array([sim, bubble]))
            if len(all_data) > 300:
                break
    print('Total bubbles included:', len(all_data))

if get_averaged:
    for cind, cth in enumerate(crit_radList):
        for tind, tsh in enumerate(crit_threshList):
            path = average_file(*exp_params)+'_critrad'+str(cth)+'_crittsh'+str(tsh)+'.npy'
            if os.path.exists(path): continue

            stacks  = stack_bubbles(all_data, win, phieq, tsh, cth, plots)
            avstack = average_stacks(stacks, win, normal, plots=False)
            #except:
            #    print('Skipped cind, tind', cind, tind)
            #    continue
            np.save(path, avstack)
            print('Done cind, tind', cind, tind)

Looking at at T, m2, sigma: 0.13 0.8049999999999999 0.2661087871323195
[ 5 10 15 20 25 30 35 40 45 50]
[1.133 1.266 1.399 1.533 1.666 1.799 1.932 2.065 2.198 2.331]
Total bubbles included: 0
0 simulations for this combination.


IndexError: list index out of range

In [ ]:
print('Looking at at T, m2, sigma:', temp, m2, sigmafld)
get_plotcomp = True

delt1 = 100

if get_plotcomp:
    varmat  = np.zeros((len(crit_radList), len(crit_threshList)))
    for cind, cth in enumerate(crit_radList):
        for tind, tsh in enumerate(crit_threshList):
            bubble = np.load(average_file(*exp_params)+'_critrad'+str(cth)+'_crittsh'+str(tsh)+'.npy')
            bubble[bubble==0.] = 'nan'

            bubble2measure = np.abs(bubble[0,0])
            nT, nN = np.shape(bubble2measure)
#             for crit_rad in crit_radList:
#                 for crit_thresh in crit_threshList:
#                     tcen, xcen = find_nucleation_center(bubble2measure, phieq, crit_thresh, crit_rad)
#                     tcen -= 35
#                     tl,tr = max(0, tcen-delt1), min(nT, tcen+delt1+1)
#                     xl,xr = max(0, xcen-delt1), min(nN, xcen+delt1+1)

#                     err_f = bubble[1, 0][tl:tr,xl:xr]  #field variance
#                     err_m = bubble[1, 1][tl:tr,xl:xr]  #momentum variance
#                     varmat[cind, tind] += np.nanmean(np.abs(err_f))
            tcen, xcen = find_nucleation_center(bubble2measure, phieq, tsh, cth)
            tcen -= 35
            tl,tr = max(0, tcen-delt1), min(nT, tcen+delt1+1)
            xl,xr = max(0, xcen-delt1), min(nN, xcen+delt1+1)

            err_f = bubble[1, 0][tl:tr,xl:xr]  #field variance
            err_m = bubble[1, 1][tl:tr,xl:xr]  #momentum variance
            varmat[cind, tind] += np.nanmean(np.abs(err_f))

    colmin, rowmin = np.where(varmat == np.nanmin(varmat))
    final_crit_rad = crit_radList[colmin][0]
    final_crit_thresh = crit_threshList[rowmin][0]

    fig, ax = plt.subplots(1,1, figsize = (3.5,3))
    ext = [crit_threshList[0], crit_threshList[-1], crit_radList[0], crit_radList[-1]]

    im0 = plt.imshow(varmat, interpolation=None, norm=LogNorm(), extent=ext, aspect='auto', origin='lower', cmap='RdBu')
    clb = plt.colorbar(im0)
    clb.ax.set_title(r'${\rm var} \, \left< \bar{\varphi} \right> $', size=11, horizontalalignment='center', verticalalignment='bottom')
    clb.ax.yaxis.set_offset_position('right')                         
    plt.plot(final_crit_thresh, final_crit_rad, color='white', marker='*')
    #plt.plot(final_crit_thresh + 0.5*(crit_threshList[1]-crit_threshList[0]), final_crit_rad + 0.5*(crit_radList[1]-crit_radList[0]), color='white', marker='*')

    xx, yy = np.meshgrid(crit_threshList, crit_radList)
    ax.contour(xx,yy,np.abs(np.log(varmat)), levels=6, aspect='auto', interpolation='gaussian', extent=ext, origin='lower', colors='k', linewidths=0.5)

    ax.set_ylabel(r'$R$')
    ax.set_xlabel(r'$\bar{\phi}$')
    beautify(ax, times=-90)
    ax.legend(title=r'$T = {:.2f}$'.format(temp), labelspacing=0., frameon='true', facecolor='white', framealpha=0.85, edgecolor='k', borderpad=0.3)
    fig.tight_layout()
    plt.savefig('./plots/residual_averaging_T'+str(temp)+'.pdf')
    plt.show()

Looking at at T, m2, sigma: 0.1 0.85 0.23030938081773686


In [ ]:
if True:
    tp = 0 # 0 for average, 1 for error
    cp = 0 # 0 - field, 1 - momentum
    delt = 100; print(delt)

    fig, ax = plt.subplots(len(crit_radList), len(crit_threshList), figsize = (len(crit_threshList)*5, len(crit_radList)*5), sharey=True, sharex=True)
    for cind, cth in enumerate(crit_radList):
        for tind, tsh in enumerate(crit_threshList):
            bubble = np.load(average_file(*exp_params)+'_critrad'+str(cth)+'_crittsh'+str(tsh)+'.npy')
            bubble[bubble==0.] = 'nan'

            bubble2measure = np.abs(bubble[tp, cp])
            if cp==0:
                bubble2measure[bubble2measure > tsh] = tsh
            try:
                tcen, xcen = find_nucleation_center(bubble2measure, phieq, tsh, cth)
            except:
                continue
            tcen-=35
            nT, nN = np.shape(bubble2measure)
            tl,tr = max(0, tcen-delt), min(nT, tcen+delt+1)
            xl,xr = max(0, xcen-delt), min(nN, xcen+delt+1)
            ext = np.array([xl-xcen,xr-xcen,tl-tcen,tr-tcen])

            bubble2plot = bubble2measure[tl:tr,xl:xr]

            im0  = ax[cind,tind].imshow(bubble2plot, extent=ext, interpolation=None, origin='lower', cmap='tab20c')
            clb0 = plt.colorbar(im0, ax = ax[cind,tind], shrink=0.6)
            lab  = r'${{\rm size}}={:.0f}'.format(cth)+r'$, {{\rm thresh}}={:.2f}'.format(tsh)
            ax[cind,tind].plot(0,0, ls=None, marker='o', color='k', label=lab)
            beautify(ax[cind,tind])
            ax[cind,tind].legend(loc=4, fancybox=True, frameon=True, framealpha=0.75, borderpad=0.3)

            if tsh == final_crit_thresh and cth == final_crit_rad:
                ax[cind,tind].plot(0., 0., color='red', ms=10, marker='*')

    plt.tight_layout()
    plt.show()

In [ ]:
rfin = np.argwhere(crit_radList == final_crit_rad)[0,0]
tfin = np.argwhere(crit_threshList == final_crit_thresh)[0,0]
crit_rad = final_crit_rad
crit_thresh = final_crit_thresh
print(rfin, tfin, crit_rad, crit_thresh)

get_final_averaged = True
if get_final_averaged:
    path = decay_times_file(*exp_params, minSim, maxSim, nTimeMAX)
    if os.path.exists(path):
        decay_times = np.load(path)

        minDecTime = 256
        alltimes   = decay_times[:,1]
        simList2Do = decay_times[alltimes>=minDecTime, 0]

    all_data = []
    fig, ax = plt.subplots(1,1, figsize = (4.5,2.5))
    for sim in simList2Do:
        path2RESTsim = rest_sim_location(*exp_params, sim)
        if os.path.exists(path2RESTsim):
            sim, bubble, totbeta = np.load(path2RESTsim, allow_pickle=True)

            real = np.copy(bubble)
            real = np.abs(real[0])
            real = gaussian_filter(real, 1, mode='nearest')
            nT, nN = np.shape(real)
            tcen, xcen = find_nucleation_center(real, phieq, crit_thresh, crit_rad)
            t, x = np.linspace(-tcen, nT-1-tcen, nT), np.linspace(-xcen, nN-1-xcen, nN)

        #    test = bubble[0, tcen, xcen-50:xcen+50]
        #    if np.nanmean(test) < 0:
        #        bubble = - bubble

            test = bubble[0, tcen, xcen-100:xcen+100]
            plt.plot(test, lw=1)
            plt.title(sim)

            all_data.append(np.array([sim, bubble]))
    print('Total bubbles included:', len(all_data))
    plt.show()

    stacks  = stack_bubbles(all_data, win, phieq, crit_thresh, crit_rad, False)
    avstack = average_stacks(stacks, win, normal, True)
    np.save(average_file(*exp_params), avstack)


In [ ]:
def plot_zoomin(real, threshold=2., winsize=100, title=None):
    real = real[0]
    nT, nN = np.shape(real)
    t_centre, x_centre = find_nucleation_center(real, phieq, crit_thresh, crit_rad)
    tl_stop, tr_stop = int(max(0, t_centre - winsize)), int(min(nT, t_centre + winsize//2))
    xl_stop, xr_stop = int(max(0, x_centre - winsize)), int(min(nN, x_centre + winsize))
    real = real[tl_stop:tr_stop, xl_stop:xr_stop]
    nT, nN = np.shape(real)
    tcen, xcen = find_nucleation_center(real, phieq, crit_thresh, crit_rad)
    t, x = np.linspace(-tcen, nT-1-tcen, nT), np.linspace(-xcen, nN-1-xcen, nN)
    cds = np.abs(real) > threshold
    real[cds] = threshold
    simple_imshow([real], x, t, title=title, contour=False, ret=False)
    return

In [ ]:
path = average_file(*exp_params)+'_critrad'+str(crit_rad)+'_crittsh'+str(crit_thresh)+'.npy'
average_bubble_filtered = np.load(path)

plot_zoomin(average_bubble_filtered[0], threshold=20, winsize=60)

average_bubble_loaded = np.load(average_file(*exp_params))

plot_zoomin(average_bubble_loaded[0], threshold=20, winsize=60)


In [ ]:
titl = [r'$\bar{\phi}$',  r'$\bar{\Pi}$']

for avbub in [average_bubble_loaded, average_bubble_filtered]:
    bubble2measure = avbub[0][0]
    nT, nN = np.shape(bubble2measure)
    tcen, xcen = find_nucleation_center(bubble2measure, phieq, crit_thresh, crit_rad)
    print(tcen, xcen)
    
    tcen-= 40
    size = 130
    tl,tr = max(0, tcen-size), min(nT, tcen+size+1)
    xl,xr = max(0, xcen-size), min(nN, xcen+size+1)
    ext = np.array([xl-xcen,xr-xcen,tl-tcen,tr-tcen])

    fig, ax = plt.subplots(1, 2, figsize = (6, 3), sharey=True)
    for cp in range(2):
        # 0 - field, 1 - momentum
        bubble2plot = np.copy(avbub[0][cp,tl:tr,xl:xr])

        im = ax[cp].imshow(bubble2plot, interpolation=None, extent=ext, origin='lower', cmap='tab20b')
        cbar = plt.colorbar(im, ax=ax[cp], shrink=0.7)
        cbar.ax.set_title(titl[cp])

        nT, nN = np.shape(bubble2plot)
        tt = np.linspace(tl-tcen, tr-tcen, nT)
        xx = np.linspace(xl-xcen, xr-xcen, nN)
        ttt1, xxx1 = np.meshgrid(tt, xx)

        lavs = [8, 6][cp]
    #  ax[cp].contour(xxx1, ttt1, bubble2plot.T, levels=lavs, aspect='auto', interpolation=None, extent=ext, origin='lower', colors='k',linewidths=0.5)
        ax[cp].set_xlabel(r'$r$')
    ax[0].set_ylabel(r'$t$')
    beautify(ax, times=-90)
    plt.tight_layout()
    plt.savefig('./plots/average_bubble'+str(temp)+'.pdf')
    plt.show()


In [ ]:
tp = 0# 0 for average, 1 for error

titl = [r'$\bar{\phi}$',  r'$\bar{\Pi}$']

for avbub in [average_bubble_loaded, average_bubble_filtered]:
    bubble2measure = avbub[0][0]
    nT, nN = np.shape(bubble2measure)
    tcen, xcen = find_nucleation_center(bubble2measure, phieq, crit_thresh, crit_rad)
    print(tcen, xcen)
    size = 100
    tl,tr = max(0, tcen-size), min(nT, tcen+size+1)
    xl,xr = max(0, xcen-size), min(nN, xcen+size+1)
    ext = np.array([xl-xcen,xr-xcen,tl-tcen,tr-tcen])

    fld_fld = np.copy(avbub[0][0])
    fld_mom = np.copy(avbub[0][1])

    fld_fld[fld_fld > crit_thresh] = crit_thresh
    fld_fld = gaussian_filter1d(fld_fld, 1, mode='nearest')
    fld_mom = gaussian_filter1d(fld_mom, 1, mode='nearest')

    fld_fld = fld_fld[tl:tr, xl:xr]
    fld_mom = fld_mom[tl:tr, xl:xr]

    kinetic   = 0.5*fld_mom**2.
    gradient  = 0.5*np.gradient(fld_fld, dx)[1]**2.
    potential = V(fld_fld)

    total = kinetic + gradient + potential
    picks = [kinetic, gradient, potential, total]
    titl = [r'$\rm Kinetic$', r'$\rm Gradient$', r'$\rm Potential$', r'$\rm Total$']

    fig, ax = plt.subplots(1, 4, figsize = (13, 3), sharey=True)
    for fi, field in enumerate(picks):

        im = ax[fi].imshow(field, extent=ext, origin='lower', cmap='tab20c')
        cbar = fig.colorbar(im, ax=ax[fi], shrink=0.7)

        nT, nN = np.shape(field)
        tt = np.linspace(tl-tcen, tr-tcen, nT)
        xx = np.linspace(xl-xcen, xr-xcen, nN)
        ttt1, xxx1 = np.meshgrid(tt, xx)
        ax[fi].contour(xxx1, ttt1, field.T, levels=8, aspect='auto', interpolation='none', extent=ext, origin='lower', colors='k', linewidths=0.5)

        ax[fi].set_title(titl[fi])
        ax[fi].set_xlabel(r'$r$')
    ax[0].set_ylabel(r'$t$')
    plt.savefig('./plots/energies.pdf')
    plt.show()
